# sys.dm\_tran\_active\_transactions:

<span style="color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Retorna informações sobre transações para a instância do SQL Server.</span>

<span style="background-color: rgb(255, 255, 255); color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Requer a permissão <b>VIEW SERVER PERFORMANCE STATE</b> no servidor.</span>

# **<mark>Query 1</mark>**

## <span style="background-color: rgb(247, 247, 248); color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">Esta consulta fornece uma visão detalhada das transações ativas no banco de dados, incluindo informações sobre o que cada transação está fazendo, seu estado, comandos em execução, bloqueios, detalhes de consulta e muito mais. Isso pode ser útil para fins de monitoramento e diagnóstico de problemas de desempenho ou bloqueios no banco de dados.</span>

In [3]:
SELECT DB_NAME(er.database_id),
       tat.transaction_id,
       tat.name,
	   er.transaction_isolation_level,
       tat.transaction_begin_time,
	   DATEDIFF(SECOND, tat.transaction_begin_time, GETDATE()) as tran_elapsed_time_seconds,
       tat.transaction_type,
       er.session_id,
       tat.transaction_state,
       tat.transaction_status,
       er.request_id,
       er.status,
       er.command,
       er.statement_end_offset,
       er.user_id,
       er.blocking_session_id,
       er.wait_type,
       er.wait_time,
       er.last_wait_type,
       er.wait_resource,
       er.open_transaction_count,
       er.percent_complete,
       er.estimated_completion_time,
       er.cpu_time,
       er.total_elapsed_time,
       er.scheduler_id,
       er.reads,
       er.writes,
       er.logical_reads,
       er.text_size,
    
       er.lock_timeout,
       er.deadlock_priority,
       er.row_count,
       er.prev_error,
       er.nest_level,
       er.granted_query_memory,
       [Query Completa] = text
  FROM sys.dm_tran_active_transactions tat
 INNER JOIN sys.dm_exec_requests er
    ON tat.transaction_id = er.transaction_id
 CROSS APPLY sys.dm_exec_sql_text(er.sql_handle) Bat
 WHERE [er].[session_id] <> @@SPID




(0 rows affected)

Total execution time: 00:00:00.005

(No column name),transaction_id,name,transaction_isolation_level,transaction_begin_time,tran_elapsed_time_seconds,transaction_type,session_id,transaction_state,transaction_status,request_id,status,command,statement_end_offset,user_id,blocking_session_id,wait_type,wait_time,last_wait_type,wait_resource,open_transaction_count,percent_complete,estimated_completion_time,cpu_time,total_elapsed_time,scheduler_id,reads,writes,logical_reads,text_size,lock_timeout,deadlock_priority,row_count,prev_error,nest_level,granted_query_memory,Query Completa


# **<mark>Query 2</mark>**

## <span style="background-color: rgb(247, 247, 248); color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">Esta consulta fornece uma visão detalhada das transações ativas no banco de dados, incluindo informações sobre o que cada transação está fazendo, seu estado, comandos em execução, bloqueios, detalhes de consulta e muito mais. Isso pode ser útil para fins de monitoramento e diagnóstico de problemas de desempenho ou bloqueios no banco de dados.</span>

In [ ]:
--Query
SELECT
  GETDATE() as now,
  DATEDIFF(SECOND, transaction_begin_time, GETDATE()) as tran_elapsed_time_seconds,
  st.session_id,
  txt.text, 
  at.*
FROM
  sys.dm_tran_active_transactions at
  INNER JOIN sys.dm_tran_session_transactions st ON st.transaction_id = at.transaction_id
  LEFT OUTER JOIN sys.dm_exec_sessions sess ON st.session_id = sess.session_id
  LEFT OUTER JOIN sys.dm_exec_connections conn ON conn.session_id = sess.session_id
    OUTER APPLY sys.dm_exec_sql_text(conn.most_recent_sql_handle)  AS txt
ORDER BY
  tran_elapsed_time_seconds DESC;

**Colunas e saidas do DMV**

| Nome da coluna | Tipo de Dados | DESCRIÇÃO |
| --- | --- | --- |
| transaction\_id | bigint | ID da transação no nível da instância, e não no nível do banco de dados. É exclusiva apenas em todos os bancos de dados em uma instância, mas não em todas as instâncias de servidor. |
| Nome | nvarchar(32) | Nome da transação. O nome será substituído se a transação for marcada e o nome marcado substituirá o nome de transação. |
| transaction\_begin\_time | datetime | Hora de início da transação. |
| transaction\_type | int | Tipo de transação.<br><br>1 = Transação de leitura/gravação<br><br>2 = Transação somente leitura<br><br>3 = Transação de sistema<br><br>4 = Transação distribuída |
| transaction\_uow | uniqueidentifier | Identificador da UOW (unidade de trabalho) da transação para transações distribuídas. O MS DTC usa o identificador UOW para trabalhar com a transação distribuída. |
| transaction\_state | int | 0 = A transação não foi completamente inicializada ainda.<br><br>1 = A transação foi inicializada mas não foi iniciada.<br><br>2 = A transação está ativa.<br><br>3 = A transação foi encerrada. Isso é usado para transações somente leitura.<br><br>4 = O processo de confirmação foi iniciado na transação distribuída. Destina-se somente a transações distribuídas. A transação distribuída ainda está ativa, mas não poderá mais ser realizado o processamento.<br><br>5 = A transação está em um estado preparado e aguardando resolução.<br><br>6 = A transação foi confirmada.<br><br>7 = A transação está sendo revertida.<br><br>8 = A transação foi revertida. |
| transaction\_status | int | Identificado apenas para fins informativos. Não há suporte. A compatibilidade futura não está garantida. |
| transaction\_status2 | int | Identificado apenas para fins informativos. Não há suporte. A compatibilidade futura não está garantida. |
| dtc\_state | int | Aplica-se a: banco de dados SQL do Azure (versão inicial por meio da [versão atual](https://learn.microsoft.com/pt-br/previous-versions/azure/ee336279(v=azure.100))).<br><br>1 = ACTIVE<br><br>2 = PREPARED<br><br>3 = COMMITTED<br><br>4 = ABORTED<br><br>5 = RECOVERED |
| dtc\_status | int | Identificado apenas para fins informativos. Não há suporte. A compatibilidade futura não está garantida. |
| dtc\_isolation\_level | int | Identificado apenas para fins informativos. Não há suporte. A compatibilidade futura não está garantida. |
| filestream\_transaction\_id | varbinary(128) | Aplica-se a: banco de dados SQL do Azure (versão inicial por meio da [versão atual](https://learn.microsoft.com/pt-br/previous-versions/azure/ee336279(v=azure.100))).<br><br>Identificado apenas para fins informativos. Não há suporte. A compatibilidade futura não está garantida. |
| pdw\_node\_id | int | Aplica-se a: Azure Synapse Analytics, Analytics Platform System (PDW)<br><br>O identificador do nó em que essa distribuição está ativada. |